In [ ]:
!pip install roboflow

In [ ]:
from roboflow import Roboflow
rf = Roboflow(api_key="2nxKEUjSzlIMWDl3hWnV")

In [ ]:
print(rf.workspace())

loading Roboflow workspace...
{
  "name": "Capstone",
  "url": "capstone-qhqkt",
  "projects": [
    "capstone-qhqkt/cucumber-znek1",
    "capstone-qhqkt/hard-hat-sample-dn68r",
    "capstone-qhqkt/pepper-ldesi",
    "capstone-qhqkt/pepper-parts",
    "capstone-qhqkt/strawberry-parts-d9fnc",
    "capstone-qhqkt/strawberry-rbo1c",
    "capstone-qhqkt/tomato-3ap6q",
    "capstone-qhqkt/tomato-parts"
  ]
}


In [ ]:
project = rf.project("pepper-ldesi")

In [ ]:
records_a7 = []
records_b3 = []
records_c7 = []

for page in project.search_all(
    class_name = "a7_pepper",
    in_dataset = True,
    fields = ["name"],
):
    records_a7.extend(page)
for page in project.search_all(
    class_name = "b3_pepper",
    in_dataset = True,
    fields = ["name"],
):
    records_b3.extend(page)
for page in project.search_all(
    class_name = "c7_pepper",
    in_dataset = True,
    fields = ["name"],
):
    records_c7.extend(page)

print(len(records_a7),len(records_b3),len(records_c7))

1354 614 225


In [ ]:
fns_a7 = []
for r in records_a7:
  fns_a7.append(r["name"])
print(len(fns_a7))
fns_b3 = []
for r in records_b3:
  fns_b3.append(r["name"])
print(len(fns_b3))
fns_c7 = []
for r in records_c7:
  fns_c7.append(r["name"])
print(len(fns_c7))

1354
614
225


In [ ]:
import random
random.shuffle(fns_a7)
random.shuffle(fns_b3)
random.shuffle(fns_c7)

In [ ]:
fns_a7 = fns_a7[:200]
fns_b3 = fns_b3[:200]
fns_c7 = fns_c7[:200]

In [ ]:
labels_path = [
    "/content/drive/MyDrive/Capstone/AIHUB-dataset/104.식물 병 유발 통합 데이터/01.데이터/2.Validation/라벨링데이터/VL1_고추/병해",
    "/content/drive/MyDrive/Capstone/AIHUB-dataset/104.식물 병 유발 통합 데이터/01.데이터/2.Validation/라벨링데이터/VL1_고추/생리장해",
    "/content/drive/MyDrive/Capstone/AIHUB-dataset/104.식물 병 유발 통합 데이터/01.데이터/2.Validation/라벨링데이터/VL1_고추/작물보호제처리반응"
]

In [ ]:
categories_list = [
    "a7_pepper",
    "b3_pepper",
    "c7_pepper"
]

In [ ]:
all_records = [fns_a7,fns_b3,fns_c7]

In [ ]:
for r in all_records:
  print(len(r))

200
200
200


In [ ]:
import os
import json
part_file_name=[
    '/content/drive/MyDrive/Capstone/AIHUB-dataset/104.식물 병 유발 통합 데이터/01.데이터/2.Validation/라벨링데이터/VL1_고추/pepper-part-a7.json',
    '/content/drive/MyDrive/Capstone/AIHUB-dataset/104.식물 병 유발 통합 데이터/01.데이터/2.Validation/라벨링데이터/VL1_고추/pepper-part-b3.json',
    '/content/drive/MyDrive/Capstone/AIHUB-dataset/104.식물 병 유발 통합 데이터/01.데이터/2.Validation/라벨링데이터/VL1_고추/pepper-part-c7.json'
]

for b,path in enumerate(labels_path):
  IBM = {
      "version": "1.0",
      "type": "localization",
      "labels": categories_list,
      "annotations":{}
  }
  idx= 0
  for img in all_records[b]:
    json_fn = img.replace('jpg','').replace('JPG','')
    json_fn += 'json'
    json_file_path = os.path.join(path,json_fn)
    with open(json_file_path,"r") as json_file:
      data = json.load(json_file)
      image_filename = data["description"]["image"]
      class_name = f"{data['annotations']['disease']}_pepper"
      parts = data["annotations"]["part"]
      value = []
      for p in parts:
        temp = {}
        temp["x"] = int(p["x"]*0.5)
        temp["y"] = int(p["y"]*0.5)
        temp["x2"] = int((p["x"]+p["w"])*0.5)
        temp["y2"] = int((p["y"]+p["h"])*0.5)
        temp["id"] = str(idx)
        idx += 1
        temp["label"] = class_name
        value.append(temp)
      IBM["annotations"][img] = value
  with open(part_file_name[b],"w") as output_json_file:
    json.dump(IBM,output_json_file, ensure_ascii=False)

In [ ]:
with open('parts-uploaded-all.txt','w') as file:
  file.write(str(all_records))

In [ ]:
train_a7 = fns_a7[:160]
valid_a7 = fns_a7[160:]
train_b3 = fns_b3[:160]
valid_b3 = fns_b3[160:]
train_c7 = fns_c7[:160]
valid_c7 = fns_c7[160:]

In [ ]:
project = rf.project("pepper-parts")

In [ ]:
dir_name_i = '/content/drive/MyDrive/Capstone/AIHUB-dataset/104.식물 병 유발 통합 데이터/01.데이터/2.Validation/원천데이터/IMG_고추/고추_병해/resized_병해'
dir_name_a = '/content/drive/MyDrive/Capstone/AIHUB-dataset/104.식물 병 유발 통합 데이터/01.데이터/2.Validation/라벨링데이터/VL1_고추/pepper-part-a7.json'
for img in train_a7:
  img_path = os.path.join(dir_name_i,img)
  project.upload(img_path,dir_name_a,split='train')
for img in valid_a7:
  img_path = os.path.join(dir_name_i,img)
  project.upload(img_path,dir_name_a,split='valid')

dir_name_i = '/content/drive/MyDrive/Capstone/AIHUB-dataset/104.식물 병 유발 통합 데이터/01.데이터/2.Validation/원천데이터/IMG_고추/고추_생리장해/resized_생리장해'
dir_name_a = '/content/drive/MyDrive/Capstone/AIHUB-dataset/104.식물 병 유발 통합 데이터/01.데이터/2.Validation/라벨링데이터/VL1_고추/pepper-part-b3.json'
for img in train_b3:
  img_path = os.path.join(dir_name_i,img)
  project.upload(img_path,dir_name_a,split='train')
for img in valid_b3:
  img_path = os.path.join(dir_name_i,img)
  project.upload(img_path,dir_name_a,split='valid')

dir_name_i = '/content/drive/MyDrive/Capstone/AIHUB-dataset/104.식물 병 유발 통합 데이터/01.데이터/2.Validation/원천데이터/IMG_고추/고추_작물보호제처리반응/resized_작물보호제처리반응'
dir_name_a = '/content/drive/MyDrive/Capstone/AIHUB-dataset/104.식물 병 유발 통합 데이터/01.데이터/2.Validation/라벨링데이터/VL1_고추/pepper-part-c7.json'
for img in train_c7:
  img_path = os.path.join(dir_name_i,img)
  project.upload(img_path,dir_name_a,split='train')
for img in valid_c7:
  img_path = os.path.join(dir_name_i,img)
  project.upload(img_path,dir_name_a,split='valid')

고추-a7랑 b3만 추가 업로드 (200장씩)

In [ ]:
#o)우선 Pepper에있는 모든 a7
#p)Pepper-part에 있는 모든 a7
#o에 있지만 p에 없는 것 리스트화
#어노테이션파일만들기

In [ ]:
records_a7 = []
for page in project.search_all(
    class_name = "a7_pepper",
    in_dataset = True,
    fields = ["name"],
):
    records_a7.extend(page)
print(len(records_a7))

1354


In [ ]:
fns_a7 = []
for r in records_a7:
  fns_a7.append(r["name"])
print(len(fns_a7))

1354


In [ ]:
project = rf.project("pepper-parts")

In [ ]:
records_pa7 = []
for page in project.search_all(
    class_name = "a7_pepper",
    in_dataset = True,
    fields = ["name"],
):
    records_pa7.extend(page)
print(len(records_pa7))

200


In [ ]:
fns_pa7 = []
for r in records_pa7:
  fns_pa7.append(r["name"])
print(len(fns_pa7))

200


In [ ]:
new_a7s = []
for a in fns_a7:
  if a not in fns_pa7:
    new_a7s.append(a)
print(new_a7s)

In [ ]:
print(len(new_a7s))

1154


In [ ]:
import random
random.shuffle(new_a7s)

In [ ]:
new_a7s = new_a7s[:200]

In [ ]:
new_a7s_train = new_a7s[:160]
new_a7s_valid = new_a7s[160:]

In [ ]:
categories_list = [
    'a7_pepper'
]

In [ ]:
import os
import json
part_file_name= '/content/drive/MyDrive/Capstone/AIHUB-dataset/104.식물 병 유발 통합 데이터/01.데이터/2.Validation/라벨링데이터/VL1_고추/new-pepper-part-a7.json'
label_path = '/content/drive/MyDrive/Capstone/AIHUB-dataset/104.식물 병 유발 통합 데이터/01.데이터/2.Validation/라벨링데이터/VL1_고추/병해'
IBM = {
    "version": "1.0",
    "type": "localization",
    "labels": categories_list,
    "annotations":{}
}
idx= 0
for img in new_a7s:
  json_fn = img.replace('jpg','').replace('JPG','')
  json_fn += 'json'
  json_file_path = os.path.join(label_path,json_fn)
  with open(json_file_path,"r") as json_file:
    data = json.load(json_file)
    image_filename = data["description"]["image"]
    class_name = f"{data['annotations']['disease']}_pepper"
    parts = data["annotations"]["part"]
    value = []
    for p in parts:
      temp = {}
      temp["x"] = int(p["x"]*0.5)
      temp["y"] = int(p["y"]*0.5)
      temp["x2"] = int((p["x"]+p["w"])*0.5)
      temp["y2"] = int((p["y"]+p["h"])*0.5)
      temp["id"] = str(idx)
      idx += 1
      temp["label"] = class_name
      value.append(temp)
    IBM["annotations"][img] = value
with open(part_file_name,"w") as output_json_file:
  json.dump(IBM,output_json_file, ensure_ascii=False)

In [ ]:
#resized 디렉터리에서 올려야함!

In [ ]:
project = rf.project("pepper-parts")

In [ ]:
dir_name_i = '/content/drive/MyDrive/Capstone/AIHUB-dataset/104.식물 병 유발 통합 데이터/01.데이터/2.Validation/원천데이터/IMG_고추/고추_병해/resized_병해'
dir_name_a = '/content/drive/MyDrive/Capstone/AIHUB-dataset/104.식물 병 유발 통합 데이터/01.데이터/2.Validation/라벨링데이터/VL1_고추/new-pepper-part-a7.json'
for img in new_a7s_train:
  img_path = os.path.join(dir_name_i,img)
  project.upload(img_path,dir_name_a,split='train')
for img in new_a7s_valid:
  img_path = os.path.join(dir_name_i,img)
  project.upload(img_path,dir_name_a,split='valid')

Duplicate image not uploaded: /content/drive/MyDrive/Capstone/AIHUB-dataset/104.식물 병 유발 통합 데이터/01.데이터/2.Validation/원천데이터/IMG_고추/고추_병해/resized_병해/557894_20210825_5_1_a7_1_2_13_1_84.jpg


/content/drive/MyDrive/Capstone/AIHUB-dataset/104.식물 병 유발 통합 데이터/01.데이터/2.Validation/라벨링데이터/VL1_고추/new-pepper-part-a7.json ERROR saving annotation: save annotation for qlTepDg34qyi3RtNGyCz / bad response: 400: {'message': 'Image was already annotated.', 'type': 'InvalidImageException', 'hint': 'This image was already annotated; to overwrite the annotation, pass overwrite=true as a query parameter.'}


Duplicate image not uploaded: /content/drive/MyDrive/Capstone/AIHUB-dataset/104.식물 병 유발 통합 데이터/01.데이터/2.Validation/원천데이터/IMG_고추/고추_병해/resized_병해/555795_20210728_5_1_a7_1_2_13_1_75.jpg


/content/drive/MyDrive/Capstone/AIHUB-dataset/104.식물 병 유발 통합 데이터/01.데이터/2.Validation/라벨링데이터/VL1_고추/new-pepper-part-a7.json ERROR saving annotation: save annotation for J1WxHIQ9WoOzEsW9bd8D / bad response: 400: {'message': 'Image was already annotated.', 'type': 'InvalidImageException', 'hint': 'This image was already annotated; to overwrite the annotation, pass overwrite=true as a query parameter.'}


Duplicate image not uploaded: /content/drive/MyDrive/Capstone/AIHUB-dataset/104.식물 병 유발 통합 데이터/01.데이터/2.Validation/원천데이터/IMG_고추/고추_병해/resized_병해/557350_20210809_5_1_a7_1_2_13_1_3.jpg


...

/content/drive/MyDrive/Capstone/AIHUB-dataset/104.식물 병 유발 통합 데이터/01.데이터/2.Validation/라벨링데이터/VL1_고추/new-pepper-part-a7.json ERROR saving annotation: save annotation for cNdrsXYTrHBkshLoWRS3 / bad response: 400: {'message': 'Image was already annotated.', 'type': 'InvalidImageException', 'hint': 'This image was already annotated; to overwrite the annotation, pass overwrite=true as a query parameter.'}


Duplicate image not uploaded: /content/drive/MyDrive/Capstone/AIHUB-dataset/104.식물 병 유발 통합 데이터/01.데이터/2.Validation/원천데이터/IMG_고추/고추_병해/resized_병해/557247_20210806_5_1_a7_1_2_13_1_19.jpg


/content/drive/MyDrive/Capstone/AIHUB-dataset/104.식물 병 유발 통합 데이터/01.데이터/2.Validation/라벨링데이터/VL1_고추/new-pepper-part-a7.json ERROR saving annotation: save annotation for 0SBSzdP5CmTssiPgbls4 / bad response: 400: {'message': 'Image was already annotated.', 'type': 'InvalidImageException', 'hint': 'This image was already annotated; to overwrite the annotation, pass overwrite=true as a query parameter.'}


Duplicate image not uploaded: /content/drive/MyDrive/Capstone/AIHUB-dataset/104.식물 병 유발 통합 데이터/01.데이터/2.Validation/원천데이터/IMG_고추/고추_병해/resized_병해/556561_20210729_5_1_a7_1_2_13_1_120.jpg


/content/drive/MyDrive/Capstone/AIHUB-dataset/104.식물 병 유발 통합 데이터/01.데이터/2.Validation/라벨링데이터/VL1_고추/new-pepper-part-a7.json ERROR saving annotation: save annotation for cNBZ1nP3xCkoZEqwYjtu / bad response: 400: {'message': 'Image was already annotated.', 'type': 'InvalidImageException', 'hint': 'This image was already annotated; to overwrite the annotation, pass overwrite=true as a query parameter.'}
